In [13]:
import os, sys
import django
sys.path.append('../') # add path to project root dir
os.environ["DJANGO_SETTINGS_MODULE"] = "flourish.settings"
from django.apps import apps as django_apps
from django.forms import forms
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"

In [14]:
from edc_appointment.models import Appointment

# Remove 2000M schedules for ANC participants
subject_identifiers = AntenatalEnrollment.objects.values_list('subject_identifier', flat=True)
for sidx in subject_identifiers:
    anc_schedules = CohortSchedules.objects.filter(
        schedule_type='antenatal',
        cohort_name='cohort_a',
        onschedule_model__startswith='flourish_caregiver')
    for schedule in anc_schedules:
        model_cls = django_apps.get_model(schedule.onschedule_model)
        try:
            anc_onsch = model_cls.objects.get(subject_identifier=sidx, schedule_name=schedule.schedule_name)
        except model_cls.DoesNotExist:
            continue
        else:
            enrol_schedules = CohortSchedules.objects.filter(
                schedule_type='enrollment',
                cohort_name='cohort_a',
                child_count=schedule.child_count)
            for enrol_schedule in enrol_schedules:
                enrol_model_cls = django_apps.get_model(enrol_schedule.onschedule_model)
                enrol_onsch = enrol_model_cls.objects.filter(
                    subject_identifier=sidx,
                    schedule_name=enrol_schedule.schedule_name)
                if enrol_onsch.exists():
                    appts = Appointment.objects.filter(subject_identifier=sidx,
                                                       schedule_name=enrol_onsch[0].schedule_name)
                    try:
                        appts.delete()
                    except Exception:
                        continue
                    else:
                        SubjectScheduleHistory.objects.filter(
                            subject_identifier=sidx,
                            schedule_name=enrol_onsch[0].schedule_name).delete()
                        enrol_onsch.delete()

In [ ]:
def create_subject_schedule_history(subject_identifier, onschedule_model, offschedule_model,
                                    schedule_name, visit_schedule_name, onschedule_datetime):
    SubjectScheduleHistory.objects.create(
        subject_identifier=subject_identifier,
        onschedule_model=onschedule_model,
        offschedule_model=offschedule_model,
        schedule_name=schedule_name,
        visit_schedule_name=visit_schedule_name,
        onschedule_datetime=onschedule_datetime,
        schedule_status='onschedule')

In [ ]:
onsch = OnScheduleCohortAAntenatal.objects.get(subject_identifier='B142-040990172-4')
create_subject_schedule_history(onsch.subject_identifier,
                                'flourish_caregiver.onschedulecohortaantenatal',
                                'flourish_caregiver.caregiveroffschedule',
                                onsch.schedule_name,
                                'a1_visit_schedule1',
                                onsch.onschedule_datetime)


In [ ]:
onsch = OnScheduleCohortAAntenatal.objects.get(subject_identifier='B142-040990639-2')
create_subject_schedule_history(onsch.subject_identifier,
                                'flourish_caregiver.onschedulecohortaantenatal',
                                'flourish_caregiver.caregiveroffschedule',
                                onsch.schedule_name,
                                'a1_visit_schedule1',
                                onsch.onschedule_datetime)


In [ ]:
from tqdm import tqdm

ultrasounds = UltraSound.objects.all()
for ultrasound in tqdm(ultrasounds):
    ultrasound.save()